<a href="https://colab.research.google.com/github/antoineor/NCAA-Kaggle-2025-Competition/blob/main/CompetNCAAVDef.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import statsmodels.api as sm


pd.set_option("display.max_column", 200)
pd.set_option("display.max_row", 1000)

In [ ]:
path = '/content/drive/MyDrive/Challenges Data/march-machine-learning-mania-2025'

# MEN

DATA LOADING

In [ ]:
#Import Data and add ratios over regular matches

tourney_results = pd.read_csv(path + '/MNCAATourneyDetailedResults.csv')
seeds = pd.read_csv(path + '/MNCAATourneySeeds.csv')
regular_results = pd.read_csv(path + '/MRegularSeasonDetailedResults.csv')

regular_results['WEFFG'] = regular_results['WFGM'] / regular_results['WFGA']
regular_results['WEFFG3'] = regular_results['WFGM3'] / regular_results['WFGA3']
regular_results['WDARE'] = regular_results['WFGM3'] / regular_results['WFGM']
regular_results['WTOQUETOQUE'] = regular_results['WAst'] / regular_results['WFGM']

regular_results['LEFFG'] = regular_results['LFGM'] / regular_results['LFGA']
regular_results['LEFFG3'] = regular_results['LFGM3'] / regular_results['LFGA3']
regular_results['LDARE'] = regular_results['LFGM3'] / regular_results['LFGM']
regular_results['LTOQUETOQUE'] = regular_results['LAst'] / regular_results['LFGM']

In [ ]:
#Function to add the mirror matches and transform W, L with T1, T2 and add a point diff feature

def prepare_data(df_data):
    df = df_data.copy()
    df.rename(columns = {'WLoc':'location'}, inplace = True )

    dfswap = df[['Season', 'DayNum', 'LTeamID', 'LScore', 'WTeamID', 'WScore', 'location', 'NumOT',
    'LFGM', 'LFGA', 'LFGM3', 'LFGA3', 'LFTM', 'LFTA', 'LOR', 'LDR', 'LAst', 'LTO', 'LStl', 'LBlk', 'LPF',
    'WFGM', 'WFGA', 'WFGM3', 'WFGA3', 'WFTM', 'WFTA', 'WOR', 'WDR', 'WAst', 'WTO', 'WStl', 'WBlk', 'WPF']]

    df.columns = df.columns.str.replace('W','T1_')
    df.columns = df.columns.str.replace('L','T2_')
    dfswap.columns = dfswap.columns.str.replace('L','T1_')
    dfswap.columns = dfswap.columns.str.replace('W','T2_')

    output = pd.concat([df, dfswap]).reset_index(drop=True)
    output.loc[output.location=='N','location'] = '0'
    output.loc[output.location=='H','location'] = '1'
    output.loc[output.location=='A','location'] = '-1'
    output.location = output.location.astype(int)
    output['PointDiff'] = output['T1_Score'] - output['T2_Score']
    return output

In [ ]:
regular_data = prepare_data(regular_results)
tourney_data = prepare_data(tourney_results)

SEASONS STATISTICS

In [ ]:
#calculate the mean of boxcore_cols by seasons for each teams

boxscore_cols = ['T1_Score', 'T2_Score',
        'T1_FGM', 'T1_FGA', 'T1_FGM3', 'T1_FGA3', 'T1_OR', 'T1_Ast', 'T1_TO', 'T1_Stl', 'T1_PF',
        'T2_FGM', 'T2_FGA', 'T2_FGM3', 'T2_FGA3', 'T2_OR', 'T2_Ast', 'T2_TO', 'T2_Stl', 'T2_Blk',
        'PointDiff', 'T1_EFFG', 'T1_EFFG3', 'T1_DARE', 'T1_TOQUETOQUE', 'T2_EFFG', 'T2_EFFG3', 'T2_DARE', 'T2_TOQUETOQUE']

funcs = [np.mean]
season_statistics = regular_data.groupby(["Season", 'T1_TeamID'])[boxscore_cols].agg(funcs).reset_index()
season_statistics.columns = [''.join(col).strip() for col in season_statistics.columns.values]
season_statistics.head(3)

<ipython-input-11-5a1491d87b65>:9: FutureWarning: The provided callable <function mean at 0x7aea46fb93a0> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  season_statistics = regular_data.groupby(["Season", 'T1_TeamID'])[boxscore_cols].agg(funcs).reset_index()


,Season,T1_TeamID,T1_Scoremean,T2_Scoremean,T1_FGMmean,T1_FGAmean,T1_FGM3mean,T1_FGA3mean,T1_ORmean,T1_Astmean,T1_TOmean,T1_Stlmean,T1_PFmean,T2_FGMmean,T2_FGAmean,T2_FGM3mean,T2_FGA3mean,T2_ORmean,T2_Astmean,T2_TOmean,T2_Stlmean,T2_Blkmean,PointDiffmean,T1_EFFGmean,T1_EFFG3mean,T1_DAREmean,T1_TOQUETOQUEmean,T2_EFFGmean,T2_EFFG3mean,T2_DAREmean,T2_TOQUETOQUEmean
0,2003,1102,57.250000,57.000000,19.142857,39.785714,7.821429,20.821429,4.178571,13.000000,11.428571,5.964286,18.750000,19.285714,42.428571,4.750000,12.428571,9.607143,9.142857,12.964286,5.428571,1.571429,0.250000,0.567934,0.449882,0.436300,0.746082,0.404633,0.314624,0.232174,0.407789
1,2003,1103,78.777778,78.148148,27.148148,55.851852,5.444444,16.074074,9.777778,15.222222,12.629630,7.259259,19.851852,27.777778,57.000000,6.666667,18.370370,12.037037,15.481481,15.333333,6.407407,2.851852,0.629630,0.542563,0.362828,0.186769,0.593458,0.457975,0.307565,0.235592,0.495248
2,2003,1104,69.285714,65.000000,24.035714,57.178571,6.357143,19.857143,13.571429,12.107143,13.285714,6.607143,18.035714,23.250000,55.500000,6.357143,19.142857,10.892857,11.678571,13.857143,5.535714,3.178571,4.285714,0.444393,0.347418,0.280960,0.551302,0.385595,0.305596,0.275764,0.489873


In [ ]:
#Functions for calculating the mean of 3 previous seasons for some features :

def get_mean_of_3_season(val_2ps, val_1ps, val_0ps,
                         weight_2=1, weight_1=2, weight_0=3,
                         degree_weight=1.0
                         ):
    weight_2 = weight_2**degree_weight
    weight_1 = weight_1**degree_weight
    weight_0 = weight_0**degree_weight
    if val_2ps == 0  and val_1ps == 0:
        return val_0ps
    elif val_2ps == 0:
        return (val_1ps*weight_2 + val_0ps*weight_1)/(weight_2 + weight_1)
    else:
        sum_of_values = val_2ps*weight_2 + val_1ps*weight_1 + val_0ps*weight_0
        return sum_of_values/(weight_2 + weight_1 + weight_0)

def get_3_feature(df_team, feature):
    value_2_seasons_ago = 0
    value_1_season_ago = 0
    value_0_season_ago = 0
    for _, val in df_team.iterrows():
        value_2_seasons_ago = value_1_season_ago
        value_1_season_ago = value_0_season_ago
        value_0_season_ago = val[feature]
    return value_2_seasons_ago, value_1_season_ago, value_0_season_ago

def write_mean_of_3_seasons(df, features, degree_weight=1.0):
    df_copy = df.copy()
    suffix = "_mn3s"
    for ft in features:
        df_copy[ft + suffix] = 0
    for idx, val in tqdm(df_copy.iterrows(), total=len(df_copy)):
        team = val.T1_TeamID
        season = val.Season
        df_team = df_copy[(df_copy.T1_TeamID == team)&
                          (df_copy.Season <= season)&
                          (df_copy.Season > season-3)]
        for ft in features:
            val_2ps, val_1ps, val_0ps = get_3_feature(df_team, ft)
            ft_mean_3 = get_mean_of_3_season(val_2ps, val_1ps, val_0ps, degree_weight=degree_weight)
            df_copy.loc[idx, ft + suffix] = ft_mean_3
    return df_copy

In [ ]:
#Calculating the mean of 3 seasons for these features

features_for_calc = ["T1_Scoremean", "T1_FGAmean",  "T1_FGA3mean"]
season_statistics_with_3_seas = write_mean_of_3_seasons(season_statistics, features_for_calc, degree_weight=1.0)
season_statistics_T1 = season_statistics_with_3_seas.copy()
season_statistics_T2 = season_statistics_with_3_seas.copy()

  0%|          | 0/7981 [00:00<?, ?it/s]<ipython-input-12-02c90eae9544>:42: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '57.25' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_copy.loc[idx, ft + suffix] = ft_mean_3
<ipython-input-12-02c90eae9544>:42: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '39.785714285714285' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_copy.loc[idx, ft + suffix] = ft_mean_3
<ipython-input-12-02c90eae9544>:42: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '20.821428571428573' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_copy.loc[idx, ft + suffix] = ft_mean_3
100%|██████████| 7981/798

In [ ]:
#Copy of season statistics for merging with right name on the column

season_statistics_T1.columns = ["T1_" + x.replace("T1_","").replace("T2_","opponent_") for x in list(season_statistics_T1.columns)]
season_statistics_T2.columns = ["T2_" + x.replace("T1_","").replace("T2_","opponent_") for x in list(season_statistics_T2.columns)]
season_statistics_T1.columns.values[0] = "Season"
season_statistics_T2.columns.values[0] = "Season"

In [ ]:
#Merging to have the new feature in the train dataset

tourney_data = pd.merge(tourney_data, season_statistics_T1, on = ['Season', 'T1_TeamID'], how = 'left')
tourney_data = pd.merge(tourney_data, season_statistics_T2, on = ['Season', 'T2_TeamID'], how = 'left')

In [ ]:
#Engineering the seed feature

seeds['seed'] = seeds['Seed'].apply(lambda x: int(x[1:3]))
seeds_T1 = seeds[['Season','TeamID','seed']].copy()
seeds_T2 = seeds[['Season','TeamID','seed']].copy()
seeds_T1.columns = ['Season','T1_TeamID','T1_seed']
seeds_T2.columns = ['Season','T2_TeamID','T2_seed']

tourney_data = pd.merge(tourney_data, seeds_T1, on = ['Season', 'T1_TeamID'], how = 'left')
tourney_data = pd.merge(tourney_data, seeds_T2, on = ['Season', 'T2_TeamID'], how = 'left')

In [ ]:
#add the label
tourney_data['T1Won'] = tourney_data['T1_Score'] > tourney_data['T2_Score']

TEAM QUALITY

In [ ]:
#Team quality : how much the ID of the team, considered as a string, can impact the prediction of victory for march madness matches

regular_season_effects = regular_data[['Season','T1_TeamID','T2_TeamID','PointDiff']].copy()
regular_season_effects['T1_TeamID'] = regular_season_effects['T1_TeamID'].astype(str)
regular_season_effects['T2_TeamID'] = regular_season_effects['T2_TeamID'].astype(str)
regular_season_effects['win'] = np.where(regular_season_effects['PointDiff']>0,1,0)

march_madness = pd.merge(seeds[['Season','TeamID']],seeds[['Season','TeamID']],on='Season')
march_madness.columns = ['Season', 'T1_TeamID', 'T2_TeamID']
march_madness.T1_TeamID = march_madness.T1_TeamID.astype(str)
march_madness.T2_TeamID = march_madness.T2_TeamID.astype(str)

#all match between teams of march madness that occured during regular season and who won
regular_season_effects = pd.merge(regular_season_effects, march_madness, on = ['Season','T1_TeamID','T2_TeamID'])

In [ ]:
regular_season_effects.tail(3)

,Season,T1_TeamID,T2_TeamID,PointDiff,win
13021,2025,1277,1458,-3,0
13022,2025,1397,1196,-9,0
13023,2025,1458,1276,-6,0


In [ ]:
#To compute the team strenght, we train a logistic regression and look at the coefficient for each teams and each seasons

def normalize_column(values):
    themean = np.mean(values)
    thestd = np.std(values)
    norm = (values - themean)/(thestd)
    return(pd.DataFrame(norm))

def team_quality(season):
    formula = 'win~-1+T1_TeamID+T2_TeamID'
    glm = sm.GLM.from_formula(formula=formula,
                              data=regular_season_effects.loc[regular_season_effects.Season==season,:],
                              family=sm.families.Binomial()).fit()
    quality = pd.DataFrame(glm.params).reset_index()
    quality.columns = ['TeamID','quality']
    quality['Season'] = season
    quality['quality'] = normalize_column(quality['quality'])
    quality['quality'] = np.exp(quality['quality'])
    quality = quality.loc[quality.TeamID.str.contains('T1_')].reset_index(drop=True)
    quality['TeamID'] = quality['TeamID'].apply(lambda x: x[10:14]).astype(int)
    print(quality['quality'].mean(), quality['quality'].std())
    return quality

glm_quality = pd.concat([team_quality(2010),
                         team_quality(2011),
                         team_quality(2012),
                         team_quality(2013),
                         team_quality(2014),
                         team_quality(2015),
                         team_quality(2016),
                         team_quality(2017),
                         team_quality(2018),
                         team_quality(2019),
                         team_quality(2021),
                         team_quality(2022),
                         team_quality(2023),
                         team_quality(2024),
                         team_quality(2025)
                        ]).reset_index(drop=True)

2.465596368202897 1.1438310235150406
2.533723606885992 0.8277687704855905
0.8979844550749319 0.5826397838204855
0.9243847418440649 0.383396103787474
2.536260688844278 1.1613680829381234
2.5318154640885915 1.9986462537640026
0.8150475571457536 0.3726513935302199
0.9951121913369343 1.6762242379484824


/usr/local/lib/python3.11/dist-packages/statsmodels/genmod/families/links.py:198: RuntimeWarning: overflow encountered in exp
  t = np.exp(-z)


1.4504222270277314 1.877734510498581
2.3792423499308204 1.216006236800427


/usr/local/lib/python3.11/dist-packages/statsmodels/genmod/families/links.py:198: RuntimeWarning: overflow encountered in exp
  t = np.exp(-z)


2.538466149039502 1.261850725899638
2.4720228654339653 1.256322792095508
0.7831527339537485 0.5203145406725298
2.3166150514730077 1.3001282499427056
2.5637447057515486 0.9172248941505184


In [ ]:
#Merging glm quality with the train dataset

glm_quality_T1 = glm_quality.copy()
glm_quality_T2 = glm_quality.copy()
glm_quality_T1.columns = ['T1_TeamID','T1_quality','Season']
glm_quality_T2.columns = ['T2_TeamID','T2_quality','Season']

tourney_data = pd.merge(tourney_data, glm_quality_T1, on = ['Season', 'T1_TeamID'], how = 'left')
tourney_data = pd.merge(tourney_data, glm_quality_T2, on = ['Season', 'T2_TeamID'], how = 'left')

tourney_data['T1_quality'] = tourney_data['T1_quality'].fillna(0.2)
tourney_data['T2_quality'] = tourney_data['T2_quality'].fillna(0.2)
tourney_data.T2_quality.isnull().sum()

np.int64(0)

SCALE WITH SEEDS FEATURE

In [ ]:
#Increase the regular features of strong teams because they don't give everything in regular seasons

features_tosws = ['T1_FGMmean','T2_FGMmean','T2_Scoremean', 'T1_Scoremean','T1_Astmean','T2_TOQUETOQUEmean','T2_FGAmean','T2_Stlmean', 'T2_Astmean']

def scale_with_seed(data, features, i, weight):
  data1 = data.copy()
  data1['weights_T1'] = np.where(data1['T1_seed']<=i, weight, 1)
  data1['weights_T2'] = np.where(data1['T2_seed']<=i, weight, 1)

  featuresT1 = [feature for feature in features_tosws if feature.startswith("T1_")]
  featuresT2 = [feature for feature in features_tosws if feature.startswith("T2_")]
  for i in featuresT1 :
    data1[i] = data1[i] * data1['weights_T1']
  for i in featuresT2 :
    data1[i] = data1[i] * data1['weights_T2']
  return data1

In [ ]:
tourney_data = scale_with_seed(tourney_data,features_tosws, 7, 1.1)

ELO RANKING

In [ ]:
#Add an elo ranking based on the previous tournament matches

rs = pd.read_csv('/content/drive/MyDrive/Challenges Data/march-machine-learning-mania-2025/MNCAATourneyCompactResults.csv')

In [ ]:
K = 20
HOME_ADVANTAGE = 100

In [ ]:
team_ids = set(rs.WTeamID).union(set(rs.LTeamID))

In [ ]:
elo_dict = dict(zip(list(team_ids), [1500] * len(team_ids)))

In [ ]:
rs['margin'] = rs.WScore - rs.LScore

In [ ]:
def elo_pred(elo1, elo2):
    return(1. / (10. ** (-(elo1 - elo2) / 400.) + 1.))

def expected_margin(elo_diff):
    return((7.5 + 0.006 * elo_diff))

def elo_update(w_elo, l_elo, margin):
    elo_diff = w_elo - l_elo
    pred = elo_pred(w_elo, l_elo)
    mult = ((margin + 3.) ** 0.8) / expected_margin(elo_diff)
    update = K * mult * (1 - pred)
    return(pred, update)

In [ ]:
preds = []
w_elo = []
l_elo = []

# Loop over all rows of the games dataframe
for row in rs.itertuples():

    # Get key data from current row
    w = row.WTeamID
    l = row.LTeamID
    margin = row.margin
    wloc = row.WLoc

    # Does either team get a home-court advantage?
    w_ad, l_ad, = 0., 0.
    if wloc == "H":
        w_ad += HOME_ADVANTAGE
    elif wloc == "A":
        l_ad += HOME_ADVANTAGE

    # Get elo updates as a result of the game
    pred, update = elo_update(elo_dict[w] + w_ad,
                              elo_dict[l] + l_ad,
                              margin)
    elo_dict[w] += update
    elo_dict[l] -= update

    # Save prediction and new Elos for each round
    preds.append(pred)
    w_elo.append(elo_dict[w])
    l_elo.append(elo_dict[l])

In [ ]:
rs['w_elo'] = w_elo
rs['l_elo'] = l_elo

In [ ]:
def final_elo_per_season(df, team_id):
    d = df.copy()
    d = d.loc[(d.WTeamID == team_id) | (d.LTeamID == team_id), :]
    d.sort_values(['Season', 'DayNum'], inplace=True)
    d.drop_duplicates(['Season'], keep='last', inplace=True)
    w_mask = d.WTeamID == team_id
    l_mask = d.LTeamID == team_id
    d['season_elo'] = None
    d.loc[w_mask, 'season_elo'] = d.loc[w_mask, 'w_elo']
    d.loc[l_mask, 'season_elo'] = d.loc[l_mask, 'l_elo']
    out = pd.DataFrame({
        'team_id': team_id,
        'season': d.Season,
        'season_elo': d.season_elo
    })
    return(out)

In [ ]:
df_list = [final_elo_per_season(rs, id) for id in team_ids]
season_elos = pd.concat(df_list)

In [ ]:
season_elos.tail(3)

,team_id,season,season_elo
2195,1463,2019,1497.662272
2347,1463,2022,1485.696337
2502,1463,2024,1474.578063


In [ ]:
def mean_elo(test1):
  T1_elo = []
  T2_elo = []
  for i in range(len(test1)):
    a = season_elos[(season_elos['team_id'] == test1['T1_TeamID'].iloc[i]) & (season_elos['season'] < test1['Season'].iloc[i])]['season_elo'].mean()
    b = season_elos[(season_elos['team_id'] == test1['T2_TeamID'].iloc[i]) & (season_elos['season'] < test1['Season'].iloc[i])]['season_elo'].mean()
    T1_elo.append(a)
    T2_elo.append(b)
  test1['T1_elo'] = T1_elo
  test1['T2_elo'] = T2_elo
  return test1


##Test en mettant des poids sur les season

w = 0.9


def weighted_mean(series: pd.Series, weight: float = 0.9) -> float:
    n = len(series)
    weights = np.array([weight ** (n - 1 - i) for i in range(n)])
    weights /= weights.sum()  # Normalize weights
    return np.sum(series * weights)


def mean_elo_weights(test1):
  test1 = mean_elo(test1)
  T1_elo = []
  T2_elo = []
  for i in range(len(test1)):
    a = weighted_mean(season_elos[(season_elos['team_id'] == test1['T1_TeamID'].iloc[i]) & (season_elos['season'] < test1['Season'].iloc[i])]['season_elo'])
    b = weighted_mean(season_elos[(season_elos['team_id'] == test1['T2_TeamID'].iloc[i]) & (season_elos['season'] < test1['Season'].iloc[i])]['season_elo'])
    T1_elo.append(a)
    T2_elo.append(b)
  test1['T1_elow'] = T1_elo
  test1['T2_elow'] = T2_elo
  return test1


In [ ]:
tourney_data = mean_elo_weights(tourney_data)

In [ ]:
tourney_data[['T1_elo', 'T2_elo']] = tourney_data[['T1_elo', 'T2_elo']].fillna(1500)
tourney_data[['T1_elow', 'T2_elow']] = tourney_data[['T1_elow', 'T2_elow']].replace(0,1500)

In [ ]:
tourney_data.tail(3)

,Season,DayNum,T1_TeamID,T1_Score,T2_TeamID,T2_Score,location,NumOT,T1_FGM,T1_FGA,T1_FGM3,T1_FGA3,T1_FTM,T1_FTA,T1_OR,T1_DR,T1_Ast,T1_TO,T1_Stl,T1_Blk,T1_PF,T2_FGM,T2_FGA,T2_FGM3,T2_FGA3,T2_FTM,T2_FTA,T2_OR,T2_DR,T2_Ast,T2_TO,T2_Stl,T2_Blk,T2_PF,PointDiff,T1_Scoremean,T1_opponent_Scoremean,T1_FGMmean,T1_FGAmean,T1_FGM3mean,T1_FGA3mean,T1_ORmean,T1_Astmean,T1_TOmean,T1_Stlmean,T1_PFmean,T1_opponent_FGMmean,T1_opponent_FGAmean,T1_opponent_FGM3mean,T1_opponent_FGA3mean,T1_opponent_ORmean,T1_opponent_Astmean,T1_opponent_TOmean,T1_opponent_Stlmean,T1_opponent_Blkmean,T1_PointDiffmean,T1_EFFGmean,T1_EFFG3mean,T1_DAREmean,T1_TOQUETOQUEmean,T1_opponent_EFFGmean,T1_opponent_EFFG3mean,T1_opponent_DAREmean,T1_opponent_TOQUETOQUEmean,T1_Scoremean_mn3s,T1_FGAmean_mn3s,T1_FGA3mean_mn3s,T2_Scoremean,T2_opponent_Scoremean,T2_FGMmean,T2_FGAmean,T2_FGM3mean,T2_FGA3mean,T2_ORmean,T2_Astmean,T2_TOmean,T2_Stlmean,T2_PFmean,T2_opponent_FGMmean,T2_opponent_FGAmean,T2_opponent_FGM3mean,T2_opponent_FGA3mean,T2_opponent_ORmean,T2_opponent_Astmean,T2_opponent_TOmean,T2_opponent_Stlmean,T2_opponent_Blkmean,T2_PointDiffmean,T2_EFFGmean,T2_EFFG3mean,T2_DAREmean,T2_TOQUETOQUEmean,T2_opponent_EFFGmean,T2_opponent_EFFG3mean,T2_opponent_DAREmean,T2_opponent_TOQUETOQUEmean,T2_Scoremean_mn3s,T2_FGAmean_mn3s,T2_FGA3mean_mn3s,T1_seed,T2_seed,T1Won,T1_quality,T2_quality,weights_T1,weights_T2,T1_elo,T2_elo,T1_elow,T2_elow
2761,2024,152,1104,72,1163,86,0,0,26,58,11,23,9,11,7,21,9,7,2,5,15,31,62,10,25,14,18,10,25,20,4,4,8,17,-14,99.825000,81.062500,33.996875,64.750000,11.062500,30.281250,11.031250,17.496875,11.812500,7.250000,19.875000,27.625000,62.687500,7.343750,23.031250,9.687500,12.406250,11.281250,7.406250,4.312500,9.687500,0.505054,0.412344,0.387450,0.544085,0.416751,0.290942,0.263648,0.417594,86.095282,63.568627,30.077819,89.617647,64.411765,31.997059,64.511765,8.794118,23.970588,10.088235,20.382353,9.117647,6.858824,16.235294,22.558824,56.676471,6.00000,18.794118,7.588235,10.382353,10.088235,5.500000,2.882353,17.058824,0.505066,0.374809,0.307358,0.713711,0.386888,0.292615,0.260506,0.445059,79.418059,59.594047,23.899751,4,1,False,2.669548,2.913214,1.1,1.1,1535.431973,1653.908608,1536.856969,1695.623495
2762,2024,152,1301,50,1345,63,0,0,21,57,5,19,3,4,6,22,10,11,8,3,13,22,55,10,25,9,10,10,28,13,14,5,2,8,-13,76.361111,72.694444,27.388889,60.972222,7.055556,20.388889,9.055556,12.694444,9.222222,7.416667,16.361111,25.916667,58.416667,7.194444,20.611111,8.472222,12.583333,11.555556,5.194444,3.111111,3.666667,0.473005,0.342557,0.255731,0.470348,0.427370,0.331121,0.275590,0.475792,76.407828,62.281408,22.303188,91.733333,70.151515,31.366667,64.233333,8.333333,20.424242,11.030303,20.233333,10.969697,6.233333,14.363636,26.212121,62.575758,7.30303,23.272727,8.090909,14.424242,9.515152,6.848485,2.272727,13.242424,0.495067,0.413950,0.292396,0.702236,0.412801,0.300530,0.270790,0.532390,79.358734,57.196970,20.967023,11,1,False,2.764297,2.873807,1.0,1.1,1538.327497,1549.456019,1542.218730,1572.173318
2763,2024,154,1345,60,1163,75,0,0,24,54,1,7,11,15,8,19,8,9,3,3,15,30,62,6,22,9,11,13,19,18,6,3,4,18,-15,91.733333,70.151515,31.366667,58.393939,8.333333,20.424242,11.030303,20.233333,10.969697,5.666667,14.363636,26.212121,62.575758,7.303030,23.272727,8.090909,14.424242,9.515152,6.848485,2.272727,13.242424,0.495067,0.413950,0.292396,0.638396,0.412801,0.300530,0.270790,0.532390,79.358734,57.196970,20.967023,89.617647,64.411765,31.997059,64.511765,8.794118,23.970588,10.088235,20.382353,9.117647,6.858824,16.235294,22.558824,56.676471,6.00000,18.794118,7.588235,10.382353,10.088235,5.500000,2.882353,17.058824,0.505066,0.374809,0.307358,0.713711,0.386888,0.292615,0.260506,0.445059,79.418059,59.594047,23.899751,1,1,False,2.873807,2.913214,1.1,1.1,1549.456019,1653.908608,1572.173318,1695.623495


MODEL EVALUATION

In [ ]:
#Function to evaluate the model on 2024 features

label = 'T1Won'

def evaluate_model(mod, data, features):
  model = mod
  data_train = data[data['Season'] != 2024]
  data_test = data[data['Season'] == 2024]
  X_train = data_train[features]
  y_train = data_train[label]
  X_test = data_test[features]
  y_test = data_test[label]
  model.fit(X_train, y_train)
  pred = model.predict(X_test)
  pred_proba = model.predict_proba(X_test)[:,1]
  acc = (pred == y_test).mean()
  brier = ((pred_proba - y_test)**2).mean()
  return f"accuracy : {acc} , brier score : {brier}"


In [ ]:
from sklearn.ensemble import RandomForestClassifier

features = ['T2_seed', 'T1_seed', 'T1_PointDiffmean', 'T2_PointDiffmean',
       'T2_Scoremean_mn3s', 'T1_FGMmean', 'T1_Scoremean_mn3s',
       'T2_FGMmean', 'T1_PFmean', 'T2_opponent_Scoremean',
       'T1_opponent_Scoremean', 'T2_Scoremean', 'T1_Scoremean',
       'T2_TOmean', 'T1_TOmean', 'T2_PFmean', 'T1_Astmean',
       'T1_opponent_ORmean', 'T2_TOQUETOQUEmean', 'T1_opponent_Stlmean',
       'T2_FGAmean_mn3s', 'T2_FGAmean', 'T1_opponent_FGMmean',
       'T1_FGAmean_mn3s', 'T2_Stlmean', 'T1_ORmean', 'T2_Astmean',
       'T2_opponent_Blkmean', 'T2_opponent_Stlmean',
       'T1_opponent_EFFGmean', 'T1_quality', 'T2_quality','T1_elo','T2_elo', 'T1_elow', 'T2_elow']

RF1 = RandomForestClassifier(
    n_estimators=100,
    max_depth=None,
    min_samples_split=2,
    min_samples_leaf=1,
    max_features="sqrt",
    bootstrap=True,
    random_state=42
)
evaluate_model(RF1,tourney_data,features)

'accuracy : 0.753731343283582 , brier score : 0.1769925373134328'

In [ ]:
#Final Fit - MEN

features = ['T2_seed', 'T1_seed', 'T1_PointDiffmean', 'T2_PointDiffmean',
       'T2_Scoremean_mn3s', 'T1_FGMmean', 'T1_Scoremean_mn3s',
       'T2_FGMmean', 'T1_PFmean', 'T2_opponent_Scoremean',
       'T1_opponent_Scoremean', 'T2_Scoremean', 'T1_Scoremean',
       'T2_TOmean', 'T1_TOmean', 'T2_PFmean', 'T1_Astmean',
       'T1_opponent_ORmean', 'T2_TOQUETOQUEmean', 'T1_opponent_Stlmean',
       'T2_FGAmean_mn3s', 'T2_FGAmean', 'T1_opponent_FGMmean',
       'T1_FGAmean_mn3s', 'T2_Stlmean', 'T1_ORmean', 'T2_Astmean',
       'T2_opponent_Blkmean', 'T2_opponent_Stlmean',
       'T1_opponent_EFFGmean', 'T1_quality', 'T2_quality','T1_elo','T2_elo', 'T1_elow', 'T2_elow']

RF_final = RandomForestClassifier(
    n_estimators=100,
    max_depth=None,
    min_samples_split=2,
    min_samples_leaf=1,
    max_features="sqrt",
    bootstrap=True,
    random_state=42)

X = tourney_data[features].copy()
y = tourney_data['T1Won'].copy()

RF_final.fit(X,y)

RandomForestClassifier(random_state=42)

In [ ]:
#FURTHER WORK
#FINE TUNE THE MODEL WITH AN EVALUATION AS THE MEAN OF ALL SEASONS AS THE TEST DATASET

# WOMEN

In [ ]:
tourney_resultsW = pd.read_csv(path + '/WNCAATourneyDetailedResults.csv')
seedsW = pd.read_csv(path + '/WNCAATourneySeeds.csv')
regular_resultsW = pd.read_csv(path + '/WRegularSeasonDetailedResults.csv')

regular_resultsW['WEFFG'] = regular_resultsW['WFGM'] / regular_resultsW['WFGA']
regular_resultsW['WEFFG3'] = regular_resultsW['WFGM3'] / regular_resultsW['WFGA3']
regular_resultsW['WDARE'] = regular_resultsW['WFGM3'] / regular_resultsW['WFGM']
regular_resultsW['WTOQUETOQUE'] = regular_resultsW['WAst'] / regular_resultsW['WFGM']

regular_resultsW['LEFFG'] = regular_resultsW['LFGM'] / regular_resultsW['LFGA']
regular_resultsW['LEFFG3'] = regular_resultsW['LFGM3'] / regular_resultsW['LFGA3']
regular_resultsW['LDARE'] = regular_resultsW['LFGM3'] / regular_resultsW['LFGM']
regular_resultsW['LTOQUETOQUE'] = regular_resultsW['LAst'] / regular_resultsW['LFGM']

In [ ]:
regular_dataW = prepare_data(regular_resultsW)
tourney_dataW = prepare_data(tourney_resultsW)

In [ ]:
season_statisticsW = regular_dataW.groupby(["Season", 'T1_TeamID'])[boxscore_cols].agg(funcs).reset_index()
season_statisticsW.columns = [''.join(col).strip() for col in season_statisticsW.columns.values]
season_statisticsW.head(3)

<ipython-input-71-5aca6ba09dfd>:1: FutureWarning: The provided callable <function mean at 0x7aea46fb93a0> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  season_statisticsW = regular_dataW.groupby(["Season", 'T1_TeamID'])[boxscore_cols].agg(funcs).reset_index()


,Season,T1_TeamID,T1_Scoremean,T2_Scoremean,T1_FGMmean,T1_FGAmean,T1_FGM3mean,T1_FGA3mean,T1_ORmean,T1_Astmean,T1_TOmean,T1_Stlmean,T1_PFmean,T2_FGMmean,T2_FGAmean,T2_FGM3mean,T2_FGA3mean,T2_ORmean,T2_Astmean,T2_TOmean,T2_Stlmean,T2_Blkmean,PointDiffmean,T1_EFFGmean,T1_EFFG3mean,T1_DAREmean,T1_TOQUETOQUEmean,T2_EFFGmean,T2_EFFG3mean,T2_DAREmean,T2_TOQUETOQUEmean
0,2010,3102,51.821429,71.785714,19.142857,53.142857,4.571429,16.071429,12.464286,10.250000,17.714286,6.035714,14.964286,27.000000,56.392857,7.714286,20.500000,12.821429,19.535714,14.392857,9.821429,5.035714,-19.964286,0.450980,0.428571,0.260870,0.565217,0.315789,0.370370,0.555556,0.722222
1,2010,3103,62.233333,59.633333,22.233333,55.266667,3.933333,12.466667,14.200000,13.433333,18.266667,8.033333,17.533333,20.966667,54.966667,4.833333,15.166667,13.633333,11.700000,18.700000,9.233333,3.800000,2.600000,0.431636,0.367010,0.180574,0.597717,0.333201,0.251383,0.214533,0.511845
2,2010,3104,64.137931,67.000000,24.724138,62.103448,4.344828,14.724138,14.482759,13.103448,19.172414,7.379310,20.413793,23.896552,60.137931,5.103448,16.517241,14.862069,12.517241,19.241379,8.758621,3.586207,-2.862069,0.470421,0.364594,0.176810,0.608955,0.359990,0.264098,0.188905,0.519286


In [ ]:
features_for_calc = ["T1_Scoremean", "T1_FGAmean",  "T1_FGA3mean"]
season_statistics_with_3_seasW = write_mean_of_3_seasons(season_statisticsW, features_for_calc, degree_weight=1.0)
season_statistics_T1W = season_statistics_with_3_seasW.copy()
season_statistics_T2W = season_statistics_with_3_seasW.copy()

  0%|          | 0/5602 [00:00<?, ?it/s]<ipython-input-12-02c90eae9544>:42: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '51.82142857142857' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_copy.loc[idx, ft + suffix] = ft_mean_3
<ipython-input-12-02c90eae9544>:42: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '53.142857142857146' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_copy.loc[idx, ft + suffix] = ft_mean_3
<ipython-input-12-02c90eae9544>:42: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '16.071428571428573' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_copy.loc[idx, ft + suffix] = ft_mean_3
100%|████████

In [ ]:
season_statistics_T1W.columns = ["T1_" + x.replace("T1_","").replace("T2_","opponent_") for x in list(season_statistics_T1W.columns)]
season_statistics_T2W.columns = ["T2_" + x.replace("T1_","").replace("T2_","opponent_") for x in list(season_statistics_T2W.columns)]
season_statistics_T1W.columns.values[0] = "Season"
season_statistics_T2W.columns.values[0] = "Season"

In [ ]:
tourney_dataW = pd.merge(tourney_dataW, season_statistics_T1W, on = ['Season', 'T1_TeamID'], how = 'left')
tourney_dataW = pd.merge(tourney_dataW, season_statistics_T2W, on = ['Season', 'T2_TeamID'], how = 'left')

In [ ]:
seedsW['seed'] = seedsW['Seed'].apply(lambda x: int(x[1:3]))
seeds_T1W = seedsW[['Season','TeamID','seed']].copy()
seeds_T2W = seedsW[['Season','TeamID','seed']].copy()
seeds_T1W.columns = ['Season','T1_TeamID','T1_seed']
seeds_T2W.columns = ['Season','T2_TeamID','T2_seed']

tourney_dataW = pd.merge(tourney_dataW, seeds_T1W, on = ['Season', 'T1_TeamID'], how = 'left')
tourney_dataW = pd.merge(tourney_dataW, seeds_T2W, on = ['Season', 'T2_TeamID'], how = 'left')

In [ ]:
tourney_dataW['T1Won'] = tourney_dataW['T1_Score'] > tourney_dataW['T2_Score']

In [ ]:
tourney_dataW

,Season,DayNum,T1_TeamID,T1_Score,T2_TeamID,T2_Score,location,NumOT,T1_FGM,T1_FGA,T1_FGM3,T1_FGA3,T1_FTM,T1_FTA,T1_OR,T1_DR,T1_Ast,T1_TO,T1_Stl,T1_Blk,T1_PF,T2_FGM,T2_FGA,T2_FGM3,T2_FGA3,T2_FTM,T2_FTA,T2_OR,T2_DR,T2_Ast,T2_TO,T2_Stl,T2_Blk,T2_PF,PointDiff,T1_Scoremean,T1_opponent_Scoremean,T1_FGMmean,T1_FGAmean,T1_FGM3mean,T1_FGA3mean,T1_ORmean,T1_Astmean,T1_TOmean,T1_Stlmean,T1_PFmean,T1_opponent_FGMmean,T1_opponent_FGAmean,T1_opponent_FGM3mean,T1_opponent_FGA3mean,T1_opponent_ORmean,T1_opponent_Astmean,T1_opponent_TOmean,T1_opponent_Stlmean,T1_opponent_Blkmean,T1_PointDiffmean,T1_EFFGmean,T1_EFFG3mean,T1_DAREmean,T1_TOQUETOQUEmean,T1_opponent_EFFGmean,T1_opponent_EFFG3mean,T1_opponent_DAREmean,T1_opponent_TOQUETOQUEmean,T1_Scoremean_mn3s,T1_FGAmean_mn3s,T1_FGA3mean_mn3s,T2_Scoremean,T2_opponent_Scoremean,T2_FGMmean,T2_FGAmean,T2_FGM3mean,T2_FGA3mean,T2_ORmean,T2_Astmean,T2_TOmean,T2_Stlmean,T2_PFmean,T2_opponent_FGMmean,T2_opponent_FGAmean,T2_opponent_FGM3mean,T2_opponent_FGA3mean,T2_opponent_ORmean,T2_opponent_Astmean,T2_opponent_TOmean,T2_opponent_Stlmean,T2_opponent_Blkmean,T2_PointDiffmean,T2_EFFGmean,T2_EFFG3mean,T2_DAREmean,T2_TOQUETOQUEmean,T2_opponent_EFFGmean,T2_opponent_EFFG3mean,T2_opponent_DAREmean,T2_opponent_TOQUETOQUEmean,T2_Scoremean_mn3s,T2_FGAmean_mn3s,T2_FGA3mean_mn3s,T1_seed,T2_seed,T1Won
0,2010,138,3124,69,3201,55,0,0,28,57,1,5,12,19,13,24,22,12,6,2,12,21,61,10,34,3,5,17,19,12,18,4,1,18,14,72.093750,56.843750,26.093750,56.343750,2.468750,8.656250,12.187500,14.687500,16.875000,6.968750,14.125000,21.125000,62.625000,5.531250,19.937500,12.687500,9.500000,15.718750,6.656250,1.875000,15.250000,0.504368,0.350263,0.092057,0.571118,0.319964,0.249555,0.265665,0.437888,72.093750,56.343750,8.656250,73.454545,60.575758,25.848485,61.575758,8.818182,24.848485,13.878788,14.636364,15.636364,10.393939,16.393939,22.939394,56.696970,4.151515,13.848485,12.242424,11.848485,19.878788,6.757576,2.606061,12.878788,0.439896,0.369088,0.342435,0.578349,0.397614,0.269271,0.171999,0.499706,73.454545,61.575758,24.848485,4,13,True
1,2010,138,3173,67,3395,66,0,0,23,59,9,26,12,19,13,34,13,16,3,10,14,22,73,8,27,14,15,18,26,8,8,8,6,22,1,70.769231,59.500000,25.884615,61.538462,5.461538,17.461538,14.730769,14.461538,16.961538,7.730769,16.576923,21.576923,58.692308,5.423077,16.846154,12.076923,11.653846,18.346154,7.615385,4.384615,11.269231,0.430773,0.326457,0.221415,0.585973,0.357068,0.307596,0.236341,0.528358,70.769231,61.538462,17.461538,72.300000,60.300000,25.833333,62.266667,6.866667,19.200000,13.500000,15.600000,16.033333,9.833333,16.233333,21.833333,59.700000,5.733333,20.166667,14.866667,13.466667,20.666667,8.466667,3.500000,12.000000,0.440677,0.366477,0.250158,0.605916,0.342404,0.259221,0.254091,0.575936,72.300000,62.266667,19.200000,8,9,True
2,2010,138,3181,72,3214,37,1,0,26,57,4,13,16,22,13,34,15,11,10,7,11,15,56,4,15,3,8,10,21,4,16,6,4,20,35,71.593750,55.062500,26.843750,63.750000,4.562500,14.468750,17.718750,14.250000,17.906250,13.375000,17.687500,18.843750,53.593750,5.312500,16.593750,12.500000,11.375000,23.218750,7.531250,3.281250,16.531250,0.440230,0.328768,0.171785,0.536230,0.331551,0.285361,0.262075,0.586411,71.593750,63.750000,14.468750,62.833333,55.133333,22.200000,60.100000,4.900000,16.100000,14.966667,11.600000,15.533333,10.100000,18.833333,19.000000,49.933333,2.866667,10.333333,12.633333,8.733333,22.133333,6.633333,2.266667,7.700000,0.391543,0.329200,0.214735,0.569006,0.355679,0.270391,0.134569,0.347931,62.833333,60.100000,16.100000,2,15,True
3,2010,138,3199,75,3256,61,1,0,25,63,3,15,22,26,20,27,13,17,8,3,21,21,62,2,20,17,22,16,21,13,16,5,4,24,14,74.466667,60.100000,26.400000,59.200000,6.100000,16.100000,14.400000,15.333333,19.033333,8.833333,17.133333,21.333333,59.566667,5.300000,18.433333,13.400000,12.233333,19.200000,8.200000,2.133333,14.366667,0.470090,0.428150,0.237734,0.581615,0.346138,0.271506,0.252169,0.560168,74.466667,59.200000,16.100000,74.258065,64.322581,27.161290,62.516129,3.935484,12.451613,14.806452,14.51

In [ ]:
regular_season_effectsW = regular_dataW[['Season','T1_TeamID','T2_TeamID','PointDiff']].copy()
regular_season_effectsW['T1_TeamID'] = regular_season_effectsW['T1_TeamID'].astype(str)
regular_season_effectsW['T2_TeamID'] = regular_season_effectsW['T2_TeamID'].astype(str)
regular_season_effectsW['win'] = np.where(regular_season_effectsW['PointDiff']>0,1,0)

march_madnessW = pd.merge(seedsW[['Season','TeamID']],seedsW[['Season','TeamID']],on='Season')
march_madnessW.columns = ['Season', 'T1_TeamID', 'T2_TeamID']
march_madnessW.T1_TeamID = march_madnessW.T1_TeamID.astype(str)
march_madnessW.T2_TeamID = march_madnessW.T2_TeamID.astype(str)

regular_season_effectsW = pd.merge(regular_season_effectsW, march_madnessW, on = ['Season','T1_TeamID','T2_TeamID'])

In [ ]:
def team_qualityW(season):
    formula = 'win~-1+T1_TeamID+T2_TeamID'
    glm = sm.GLM.from_formula(formula=formula,
                              data=regular_season_effectsW.loc[regular_season_effectsW.Season==season,:],
                              family=sm.families.Binomial()).fit()
    quality = pd.DataFrame(glm.params).reset_index()
    quality.columns = ['TeamID','quality']
    quality['Season'] = season
    quality['quality'] = normalize_column(quality['quality'])
    quality['quality'] = np.exp(quality['quality'])
    quality = quality.loc[quality.TeamID.str.contains('T1_')].reset_index(drop=True)
    quality['TeamID'] = quality['TeamID'].apply(lambda x: x[10:14]).astype(int)
    print(quality['quality'].mean(), quality['quality'].std())
    return quality

glm_qualityW = pd.concat([team_qualityW(2010),
                         team_qualityW(2011),
                         team_qualityW(2012),
                         team_qualityW(2013),
                         team_qualityW(2014),
                         team_qualityW(2015),
                         team_qualityW(2016),
                         team_qualityW(2017),
                         team_qualityW(2018),
                         team_qualityW(2019),
                         team_qualityW(2021),
                         team_qualityW(2022),
                         team_qualityW(2023),
                         team_qualityW(2024),
                         team_qualityW(2025)
                        ]).reset_index(drop=True)

0.9962327093932201 0.9690510930127465
0.8204741108742641 0.37587543604225465


/usr/local/lib/python3.11/dist-packages/statsmodels/genmod/families/links.py:198: RuntimeWarning: overflow encountered in exp
  t = np.exp(-z)


2.521519485010218 2.075853182691188
2.602115228646059 1.2454937008447968
2.479796045700845 1.3310124206432044


/usr/local/lib/python3.11/dist-packages/statsmodels/genmod/families/links.py:198: RuntimeWarning: overflow encountered in exp
  t = np.exp(-z)


2.6553997627636634 0.7630568250673612
2.6999213415810637 0.5907784764899748
2.59167501511776 1.120068313024485
2.5707807265734592 1.0827963993943335
2.656350398352265 0.768464335625904


/usr/local/lib/python3.11/dist-packages/statsmodels/genmod/families/links.py:198: RuntimeWarning: overflow encountered in exp
  t = np.exp(-z)


1.013625909876363 0.4165951582649925
0.7982870067236976 0.32608956557746244
1.070618534071474 0.9401053102833095
1.0068696989943828 0.8124799043275561
0.7843790786901674 0.39243453916331333


In [ ]:
glm_quality_T1W = glm_qualityW.copy()
glm_quality_T2W = glm_qualityW.copy()
glm_quality_T1W.columns = ['T1_TeamID','T1_quality','Season']
glm_quality_T2W.columns = ['T2_TeamID','T2_quality','Season']

tourney_dataW = pd.merge(tourney_dataW, glm_quality_T1W, on = ['Season', 'T1_TeamID'], how = 'left')
tourney_dataW = pd.merge(tourney_dataW, glm_quality_T2W, on = ['Season', 'T2_TeamID'], how = 'left')

tourney_dataW['T1_quality'] = tourney_dataW['T1_quality'].fillna(0.2)
tourney_dataW['T2_quality'] = tourney_dataW['T2_quality'].fillna(0.2)
tourney_dataW.T2_quality.isnull().sum()

np.int64(0)

In [ ]:
tourney_dataW = scale_with_seed(tourney_dataW,features_tosws, 7, 1.1)

In [ ]:
rsW = pd.read_csv('/content/drive/MyDrive/Challenges Data/march-machine-learning-mania-2025/WNCAATourneyCompactResults.csv')

In [ ]:
K = 20
HOME_ADVANTAGE = 100

In [ ]:
team_idsW = set(rsW.WTeamID).union(set(rsW.LTeamID))

In [ ]:
elo_dictW = dict(zip(list(team_idsW), [1500] * len(team_idsW)))

In [ ]:
rsW['margin'] = rsW.WScore - rsW.LScore

In [ ]:
predsW = []
w_eloW = []
l_eloW = []

# Loop over all rows of the games dataframe
for row in rsW.itertuples():

    # Get key data from current row
    w = row.WTeamID
    l = row.LTeamID
    margin = row.margin
    wloc = row.WLoc

    # Does either team get a home-court advantage?
    w_ad, l_ad, = 0., 0.
    if wloc == "H":
        w_ad += HOME_ADVANTAGE
    elif wloc == "A":
        l_ad += HOME_ADVANTAGE

    # Get elo updates as a result of the game
    pred, update = elo_update(elo_dictW[w] + w_ad,
                              elo_dictW[l] + l_ad,
                              margin)
    elo_dictW[w] += update
    elo_dictW[l] -= update

    # Save prediction and new Elos for each round
    predsW.append(pred)
    w_eloW.append(elo_dictW[w])
    l_eloW.append(elo_dictW[l])

In [ ]:
rsW['w_elo'] = w_eloW
rsW['l_elo'] = l_eloW

In [ ]:
df_listW = [final_elo_per_season(rsW, id) for id in team_idsW]
season_elosW = pd.concat(df_listW)

In [ ]:
season_elosW

,team_id,season,season_elo
1339,3101,2019,1493.733234
1019,3103,2014,1492.742309
51,3104,1998,1505.374958
100,3104,1999,1508.214604
1429,3104,2021,1506.662594
...,...,...,...
813,3462,2010,1518.593735
863,3462,2011,1515.971499
36,3464,1998,1501.665106
136,3464,2000,1490.878528


In [ ]:
def mean_eloW(test1):
  T1_elo = []
  T2_elo = []
  for i in range(len(test1)):
    a = season_elosW[(season_elosW['team_id'] == test1['T1_TeamID'].iloc[i]) & (season_elosW['season'] < test1['Season'].iloc[i])]['season_elo'].mean()
    b = season_elosW[(season_elosW['team_id'] == test1['T2_TeamID'].iloc[i]) & (season_elosW['season'] < test1['Season'].iloc[i])]['season_elo'].mean()
    T1_elo.append(a)
    T2_elo.append(b)
  test1['T1_elo'] = T1_elo
  test1['T2_elo'] = T2_elo
  return test1


def mean_elo_weightsW(test1):
  test1 = mean_eloW(test1)
  T1_elo = []
  T2_elo = []
  for i in range(len(test1)):
    a = weighted_mean(season_elosW[(season_elosW['team_id'] == test1['T1_TeamID'].iloc[i]) & (season_elosW['season'] < test1['Season'].iloc[i])]['season_elo'])
    b = weighted_mean(season_elosW[(season_elosW['team_id'] == test1['T2_TeamID'].iloc[i]) & (season_elosW['season'] < test1['Season'].iloc[i])]['season_elo'])
    T1_elo.append(a)
    T2_elo.append(b)
  test1['T1_elow'] = T1_elo
  test1['T2_elow'] = T2_elo
  return test1

In [ ]:
tourney_dataW = mean_elo_weightsW(tourney_dataW)

In [ ]:
tourney_dataW[['T1_elo', 'T2_elo']] = tourney_dataW[['T1_elo', 'T2_elo']].fillna(1500)
tourney_dataW[['T1_elow', 'T2_elow']] = tourney_dataW[['T1_elow', 'T2_elow']].replace(0,1500)

In [ ]:
features = ['T2_seed', 'T1_seed', 'T1_PointDiffmean', 'T2_PointDiffmean',
       'T2_Scoremean_mn3s', 'T1_FGMmean', 'T1_Scoremean_mn3s',
       'T2_FGMmean', 'T1_PFmean', 'T2_opponent_Scoremean',
       'T1_opponent_Scoremean', 'T2_Scoremean', 'T1_Scoremean',
       'T2_TOmean', 'T1_TOmean', 'T2_PFmean', 'T1_Astmean',
       'T1_opponent_ORmean', 'T2_TOQUETOQUEmean', 'T1_opponent_Stlmean',
       'T2_FGAmean_mn3s', 'T2_FGAmean', 'T1_opponent_FGMmean',
       'T1_FGAmean_mn3s', 'T2_Stlmean', 'T1_ORmean', 'T2_Astmean',
       'T2_opponent_Blkmean', 'T2_opponent_Stlmean',
       'T1_opponent_EFFGmean', 'T1_quality', 'T2_quality','T1_elo','T2_elo', 'T1_elow', 'T2_elow']

RF1 = RandomForestClassifier(
    n_estimators=100,
    max_depth=None,
    min_samples_split=2,
    min_samples_leaf=1,
    max_features="sqrt",
    bootstrap=True,
    random_state=42
)
evaluate_model(RF1,tourney_dataW,features)

'accuracy : 0.8059701492537313 , brier score : 0.14825000000000002'

FINAL FIT FOR WOMEN

In [ ]:
features = ['T2_seed', 'T1_seed', 'T1_PointDiffmean', 'T2_PointDiffmean',
       'T2_Scoremean_mn3s', 'T1_FGMmean', 'T1_Scoremean_mn3s',
       'T2_FGMmean', 'T1_PFmean', 'T2_opponent_Scoremean',
       'T1_opponent_Scoremean', 'T2_Scoremean', 'T1_Scoremean',
       'T2_TOmean', 'T1_TOmean', 'T2_PFmean', 'T1_Astmean',
       'T1_opponent_ORmean', 'T2_TOQUETOQUEmean', 'T1_opponent_Stlmean',
       'T2_FGAmean_mn3s', 'T2_FGAmean', 'T1_opponent_FGMmean',
       'T1_FGAmean_mn3s', 'T2_Stlmean', 'T1_ORmean', 'T2_Astmean',
       'T2_opponent_Blkmean', 'T2_opponent_Stlmean',
       'T1_opponent_EFFGmean', 'T1_quality', 'T2_quality','T1_elo','T2_elo', 'T1_elow', 'T2_elow']

RF_finalW = RandomForestClassifier(
    n_estimators=100,
    max_depth=None,
    min_samples_split=2,
    min_samples_leaf=1,
    max_features="sqrt",
    bootstrap=True,
    random_state=42)

XW = tourney_dataW[features].copy()
yW = tourney_dataW['T1Won'].copy()

RF_finalW.fit(XW,yW)

RandomForestClassifier(random_state=42)

# SUBMISSION

In [ ]:
sub = pd.read_csv(path + '/SampleSubmissionStage2.csv')

In [ ]:
sub

,ID,Pred
0,2025_1101_1102,0.5
1,2025_1101_1103,0.5
2,2025_1101_1104,0.5
3,2025_1101_1105,0.5
4,2025_1101_1106,0.5
...,...,...
131402,2025_3477_3479,0.5
131403,2025_3477_3480,0.5
131404,2025_3478_3479,0.5
131405,2025_3478_3480,0.5


In [ ]:
subM = sub[sub['ID'].apply(lambda x: x[5:9]).astype(int) < 3000]
subW = sub[sub['ID'].apply(lambda x: x[5:9]).astype(int) > 2999]

In [ ]:
subW

,ID,Pred
66066,2025_3101_3102,0.5
66067,2025_3101_3103,0.5
66068,2025_3101_3104,0.5
66069,2025_3101_3105,0.5
66070,2025_3101_3106,0.5
...,...,...
131402,2025_3477_3479,0.5
131403,2025_3477_3480,0.5
131404,2025_3478_3479,0.5
131405,2025_3478_3480,0.5


In [ ]:
def preparing_submission(df):
  data = df.copy()
  data["Season"] = data["ID"].apply(lambda x: x[0:4]).astype(int)
  data["T1_TeamID"] = data["ID"].apply(lambda x: x[5:9]).astype(int)
  data["T2_TeamID"] = data["ID"].apply(lambda x: x[10:14]).astype(int)
  data = pd.merge(data, season_statistics_T1, on = ['Season', 'T1_TeamID'], how = 'left')
  data = pd.merge(data, season_statistics_T2, on = ['Season', 'T2_TeamID'], how = 'left')
  data = pd.merge(data, seeds_T1, on = ['Season', 'T1_TeamID'], how = 'left')
  data = pd.merge(data, seeds_T2, on = ['Season', 'T2_TeamID'], how = 'left')
  data = pd.merge(data, glm_quality_T1, on = ['Season', 'T1_TeamID'], how = 'left')
  data = pd.merge(data, glm_quality_T2, on = ['Season', 'T2_TeamID'], how = 'left')
  data['T1_quality'] = data['T1_quality'].fillna(0.2)
  data['T2_quality'] = data['T2_quality'].fillna(0.2)
  data = scale_with_seed(data,features_tosws, 7, 1.1)
  data = mean_elo_weights(data)
  data[['T1_elo', 'T2_elo']] = data[['T1_elo', 'T2_elo']].fillna(1500)
  data[['T1_elow', 'T2_elow']] = data[['T1_elow', 'T2_elow']].replace(0,1500)
  return data

def making_submission(df):
  sub_data = df[['ID']].copy()
  X = df[features].copy()
  preds = RF_final.predict_proba(X)
  sub_data['Pred'] = preds[:,1]
  return sub_data


In [ ]:
subMdf = preparing_submission(subM)

In [ ]:
#fillna with arbitrary value = 0 because all the nan are regaring matchup that will not happen
subMdf.fillna(0, inplace = True)

In [ ]:
subMdf

,ID,Pred,Season,T1_TeamID,T2_TeamID,T1_Scoremean,T1_opponent_Scoremean,T1_FGMmean,T1_FGAmean,T1_FGM3mean,T1_FGA3mean,T1_ORmean,T1_Astmean,T1_TOmean,T1_Stlmean,T1_PFmean,T1_opponent_FGMmean,T1_opponent_FGAmean,T1_opponent_FGM3mean,T1_opponent_FGA3mean,T1_opponent_ORmean,T1_opponent_Astmean,T1_opponent_TOmean,T1_opponent_Stlmean,T1_opponent_Blkmean,T1_PointDiffmean,T1_EFFGmean,T1_EFFG3mean,T1_DAREmean,T1_TOQUETOQUEmean,T1_opponent_EFFGmean,T1_opponent_EFFG3mean,T1_opponent_DAREmean,T1_opponent_TOQUETOQUEmean,T1_Scoremean_mn3s,T1_FGAmean_mn3s,T1_FGA3mean_mn3s,T2_Scoremean,T2_opponent_Scoremean,T2_FGMmean,T2_FGAmean,T2_FGM3mean,T2_FGA3mean,T2_ORmean,T2_Astmean,T2_TOmean,T2_Stlmean,T2_PFmean,T2_opponent_FGMmean,T2_opponent_FGAmean,T2_opponent_FGM3mean,T2_opponent_FGA3mean,T2_opponent_ORmean,T2_opponent_Astmean,T2_opponent_TOmean,T2_opponent_Stlmean,T2_opponent_Blkmean,T2_PointDiffmean,T2_EFFGmean,T2_EFFG3mean,T2_DAREmean,T2_TOQUETOQUEmean,T2_opponent_EFFGmean,T2_opponent_EFFG3mean,T2_opponent_DAREmean,T2_opponent_TOQUETOQUEmean,T2_Scoremean_mn3s,T2_FGAmean_mn3s,T2_FGA3mean_mn3s,T1_seed,T2_seed,T1_quality,T2_quality,weights_T1,weights_T2,T1_elo,T2_elo,T1_elow,T2_elow
0,2025_1101_1102,0.5,2025,1101,1102,67.551724,71.000000,24.206897,56.241379,4.103448,14.206897,8.655172,12.482759,14.206897,10.034483,21.448276,23.655172,51.310345,5.448276,16.689655,8.758621,12.206897,16.034483,7.896552,4.689655,-3.448276,0.481963,0.354834,0.180788,0.492261,0.411128,0.298650,0.237314,0.460635,69.297161,57.210640,15.288932,61.937500,73.656250,21.250000,50.500000,8.093750,24.468750,5.937500,13.375000,12.187500,5.656250,17.843750,26.406250,54.937500,7.218750,19.906250,7.937500,12.843750,9.687500,6.812500,3.218750,-11.718750,0.523892,0.452429,0.448852,0.759036,0.403571,0.229762,0.195169,0.449876,64.178763,51.018145,24.191868,0.0,0.0,0.2,0.200000,1.0,1.0,1490.795661,1492.363815,1490.679365,1492.226169
1,2025_1101_1103,0.5,2025,1101,1103,67.551724,71.000000,24.206897,56.241379,4.103448,14.206897,8.655172,12.482759,14.206897,10.034483,21.448276,23.655172,51.310345,5.448276,16.689655,8.758621,12.206897,16.034483,7.896552,4.689655,-3.448276,0.481963,0.354834,0.180788,0.492261,0.411128,0.298650,0.237314,0.460635,69.297161,57.210640,15.288932,83.968750,75.906250,30.218750,64.125000,10.656250,29.093750,9.968750,17.687500,11.750000,7.562500,18.281250,26.687500,61.375000,7.437500,23.343750,8.843750,11.937500,12.000000,7.187500,2.437500,8.062500,0.491428,0.373764,0.360045,0.601083,0.421924,0.311025,0.298077,0.445478,78.459173,60.419019,26.547883,0.0,13.0,0.2,1.000000,1.0,1.0,1490.795661,1462.301799,1490.679365,1458.710468
2,2025_1101_1104,0.5,2025,1101,1104,67.551724,71.000000,24.206897,56.241379,4.103448,14.206897,8.655172,12.482759,14.206897,10.034483,21.448276,23.655172,51.310345,5.448276,16.689655,8.758621,12.206897,16.034483,7.896552,4.689655,-3.448276,0.481963,0.354834,0.180788,0.492261,0.411128,0.298650,0.237314,0.460635,69.297161,57.210640,15.288932,100.233333,81.424242,34.200000,70.866667,10.424242,29.787879,10.969697,18.466667,12.121212,6.666667,19.030303,28.909091,68.090909,7.363636,23.909091,10.333333,12.666667,10.060606,7.818182,4.242424,9.696970,0.496518,0.358884,0.342975,0.601588,0.410331,0.279536,0.242537,0.409467,89.506684,64.163102,29.948474,0.0,2.0,0.2,3.110915,1.0,1.1,1490.795661,1537.906416,1490.679365,1542.328575
3,2025_1101_1105,0.5,2025,1101,1105,67.551724,71.000000,24.206897,56.241379,4.103448,14.206897,8.655172,12.482759,14.206897,10.034483,21.448276,23.655172,51.310345,5.448276,16.689655,8.758621,12.206897,16.034483,7.896552,4.689655,-3.448276,0.481963,0.354834,0.180788,0.492261,0.411128,0.298650,0.237314,0.460635,69.297161,57.210640,15.288932,69.310345,79.896552,23.137931,59.793103,7.655172,24.862069,11.344828,12.344828,15.103448,8.000000,21.310345,26.551724,56.517241,8.275862,21.206897,9.724138,15.344828,14.137931,9.068966,3.034483,-10.586207,0.459266,0.329867,0.276006,0.537193,0.430813,0.322625,0.277816,0.521472,68.881940,57.555643,20.43

In [ ]:
final_subM = making_submission(subMdf)

In [ ]:
final_subM

,ID,Pred
0,2025_1101_1102,0.49
1,2025_1101_1103,0.59
2,2025_1101_1104,0.36
3,2025_1101_1105,0.41
4,2025_1101_1106,0.49
...,...,...
66061,2025_1477_1479,0.48
66062,2025_1477_1480,0.50
66063,2025_1478_1479,0.44
66064,2025_1478_1480,0.47


In [ ]:
def preparing_submissionW(df):
  data = df.copy()
  data["Season"] = data["ID"].apply(lambda x: x[0:4]).astype(int)
  data["T1_TeamID"] = data["ID"].apply(lambda x: x[5:9]).astype(int)
  data["T2_TeamID"] = data["ID"].apply(lambda x: x[10:14]).astype(int)
  data = pd.merge(data, season_statistics_T1W, on = ['Season', 'T1_TeamID'], how = 'left')
  data = pd.merge(data, season_statistics_T2W, on = ['Season', 'T2_TeamID'], how = 'left')
  data = pd.merge(data, seeds_T1W, on = ['Season', 'T1_TeamID'], how = 'left')
  data = pd.merge(data, seeds_T2W, on = ['Season', 'T2_TeamID'], how = 'left')
  data = pd.merge(data, glm_quality_T1W, on = ['Season', 'T1_TeamID'], how = 'left')
  data = pd.merge(data, glm_quality_T2W, on = ['Season', 'T2_TeamID'], how = 'left')
  data['T1_quality'] = data['T1_quality'].fillna(0.2)
  data['T2_quality'] = data['T2_quality'].fillna(0.2)
  data = scale_with_seed(data,features_tosws, 7, 1.1)
  data = mean_elo_weightsW(data)
  data[['T1_elo', 'T2_elo']] = data[['T1_elo', 'T2_elo']].fillna(1500)
  data[['T1_elow', 'T2_elow']] = data[['T1_elow', 'T2_elow']].replace(0,1500)
  return data

def making_submissionW(df):
  sub_data = df[['ID']].copy()
  X = df[features].copy()
  preds = RF_finalW.predict_proba(X)
  sub_data['Pred'] = preds[:,1]
  return sub_data

In [ ]:
subWdf = preparing_submissionW(subW)

In [ ]:
subWdf.fillna(0, inplace = True)

In [ ]:
final_subW = making_submissionW(subWdf)

In [ ]:
final_subW

,ID,Pred
0,2025_3101_3102,0.52
1,2025_3101_3103,0.54
2,2025_3101_3104,0.25
3,2025_3101_3105,0.48
4,2025_3101_3106,0.56
...,...,...
65336,2025_3477_3479,0.46
65337,2025_3477_3480,0.44
65338,2025_3478_3479,0.43
65339,2025_3478_3480,0.44


FINAL SUBMISSION

In [ ]:
final_sub = pd.concat([final_subM, final_subW], axis = 0)

In [ ]:
final_sub

,ID,Pred
0,2025_1101_1102,0.49
1,2025_1101_1103,0.59
2,2025_1101_1104,0.36
3,2025_1101_1105,0.41
4,2025_1101_1106,0.49
...,...,...
65336,2025_3477_3479,0.46
65337,2025_3477_3480,0.44
65338,2025_3478_3479,0.43
65339,2025_3478_3480,0.44


In [ ]:
final_sub.to_csv('finalsub1_AO_RP.csv', index=False)

In [ ]:
final_sub

,ID,Pred
0,2025_1101_1102,0.49
1,2025_1101_1103,0.59
2,2025_1101_1104,0.36
3,2025_1101_1105,0.41
4,2025_1101_1106,0.49
...,...,...
65336,2025_3477_3479,0.46
65337,2025_3477_3480,0.44
65338,2025_3478_3479,0.43
65339,2025_3478_3480,0.44
